# Preparing Management Accounts
Starting with a YTD TTB you need to add the begining of the year trial balance.    

Note that during year during the first months we will not have the accountants adjustments and at some point we will.  At that point there will be a discontinuity in the accounting record.   

If you are preparing part way through the year then you may take a mid year point as the starting point for the preperation of the accounts.  This happend in 2014-2015 and September 2015 was used as a starting point or the subsequent months of the year.

In the first instance this starting point would be the January of the preceding financial year.


## Get the Jan 15 accounts

In [1]:
import datetime as dt
import math
import pandas as pd
import pickle
import sqlite3

from luca import SageData, TransactionalTrialBalance, p, PeriodReport, ExcelManagementReport,\
    LoadDatabaseError, LoadDatabase

In [2]:
rep = PeriodReport(dt.datetime(2015, 1, 1), "historic_trial_balances.db")

In [3]:
emr = ExcelManagementReport()
emr.create_one_report(rep)
rep.period_ytd.to_series().head()

10        9926.00
1001    362381.70
1100    303456.80
1102         0.00
1103    -30274.58
dtype: object

In [4]:
for attr in ('period_mtd', 'period_ytd', 'prior_mtd', 'prior_ytd'):
    print('Sum for {} = {}'.format(attr, getattr(rep, attr).sum()))

Sum for period_mtd = -75044.01
Sum for period_ytd = 0.00
Sum for prior_mtd = -107345.59
Sum for prior_ytd = 0.00


## Convert YTD for prior year to start of year
We need to close out the year by clearning the P&L and adding to the retained profit for the year.  (Note this doesn't have the accountants changes which we will need for Jan 16).   
rep.period_ytd  --> to year_start.   
Sum > 3000 and add to retained profit 2125
Clear all > 3000

In [5]:
start_year=rep.period_ytd.close_period()
start_year.to_series().head()

P&L = -45685.49
Prev retain = -509969.50, changing to -555654.99
New retained profit = -555654.99
Sum before clearing old balances = -45685.49
Sum after clearing old balances = 0.00


10        9926.00
1001    362381.70
1100    303456.80
1102         0.00
1103    -30274.58
dtype: object

## Add start of year to TTB
In order to create the accounts for the YTD we need to add the TTB for the YTD to the year start.   
This should be a generalised TB add.

In [6]:
sage_2015 = pickle.load(open('sage2015.pickle', 'rb'))

In [7]:
ttb = sage_2015.transactional_trial_balance(dt.date(2015, 2, 1), dt.date(2015, 2, 28))
print('Sum ttb {}'.format(ttb.sum()))
ttb.to_series().head()

Sum ttb 0.00


1001     25646.21
1100     60958.94
1120       468.84
1200    -70576.88
1205         0.00
dtype: object

In [8]:
ttb_converter = TransactionalTrialBalance(start_year.chart_of_accounts)

In [10]:
ytd = start_year + ttb_converter.convert_trial_balance(ttb)

# Create account for the month
Having created YTD and MTD then upload into the database under temporary notation
Then load a report with the correct data and run the report.

In [11]:
ttb_converter = TransactionalTrialBalance(start_year.chart_of_accounts)
with LoadDatabase('historic_trial_balances.db') as ld:
    ttb = sage_2015.transactional_trial_balance(dt.date(2015, 2, 1), dt.date(2015, 2, 28))
    ytd = start_year + ttb_converter.convert_trial_balance(ttb)
    ld.load_tb_to_database(ytd, 'AYTD-FEB-15', overwrite=True)
    # TODO the signs of this are wrong it should be negative for 2107 so that in the BS
    # it comes out as positive

### Report on the new data

In [12]:
new_rep = PeriodReport(dt.datetime(2015, 2, 1), "historic_trial_balances.db", 
                       period_ytd_prefix="AYTD")

In [14]:
emr.create_one_report(new_rep)
new_rep.prior_mtd.to_series().tail()

8430    678.98
8433      0.00
8435    311.91
8440    860.04
9999      0.00
dtype: object

In [ ]:
print(mgmt_tb['TB'].sum())
print(mgmt_tb.ix[10])

## Create accounts for the year
Iterate for the whole year.

## Using Jan 16 accounts generate accounts for 2016-2017
This can be redone once we have finalised the Jan 16 accounts

# Playpit


In [ ]:
new_rep.period_ytd['TB']